##Paquetes


In [ ]:
#Instalar los paquetes RDKit y MolVS para la estandarización
%%capture
!pip install rdkit
!pip install molvs

In [ ]:
#Importar los módulos de cada paquete necesarios para todas las operaciones
import os
import io
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import PandasTools
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.rdmolops import GetFormalCharge, RemoveStereochemistry
from molvs.standardize import Standardizer
from molvs.charge import Uncharger, Reionizer
from molvs.fragment import LargestFragmentChooser
from molvs.tautomer import TautomerCanonicalizer

ModuleNotFoundError: No module named 'rdkit'

##Carga del CSV

In [ ]:
df = pd.read_csv('file.csv', sep = ',') #Subir el archivo al entorno y cambiar el nombre y el separador según sea necesario
df.head() #Revisar las primeras filas para comprobar que se cargó correctamente

,Unnamed: 0,NAME,SMILES,SOURCE_SPECIES,YEAR,REFERENCE
0,0,Gemin D,O=C(OC(C(=O)CO)C1OC(=O)c2cc(O)c(O)c(O)c2-c2c(c...,Phyllanthus acuminatus,2021.0,https://doi.org/10.3390/molecules26216493
1,1,Phyllanemblinin B,O=C(OC1OC(CO)C2OC(=O)c3cc(O)c(O)c(O)c3-c3c(cc(...,Phyllanthus acuminatus,2021.0,https://doi.org/10.3390/molecules26216493
2,2,Corilagin,O=C(OC1OC2COC(=O)c3cc(O)c(O)c(O)c3-c3c(cc(O)c(...,Phyllanthus acuminatus,2021.0,https://doi.org/10.3390/molecules26216493
3,3,Prodelphinidin B dimer,Oc1cc(O)c2c(c1)OC(c1cc(O)c(O)c(O)c1)C(O)C2c1c(...,Phyllanthus acuminatus,2021.0,https://doi.org/10.3390/molecules26216493
4,4,(epi)galocatequina,Oc1cc(O)c2c(c1)OC(c1cc(O)c(O)c(O)c1)C(O)C2,Phyllanthus acuminatus,2021.0,https://doi.org/10.3390/molecules26216493


##Estandarización
Adaptado del curso "Herramientas quimioinformáticas para el diseño de fármacos", tomado del [GitHub de DIFACQUIM](https://github.com/DIFACQUIM/Cursos/blob/main/5_2_Curado_de_bases_de_datos_versi%C3%B3n2.ipynb)

In [ ]:
#Definir funciones
STD = Standardizer() #Devuelve la versión estandarizada (canónica) de un SMILES dado
LFC = LargestFragmentChooser() #Mantiene el fragmento más grande de una sal (compuestos iónicos)
UC = Uncharger() #Correcciones de carga, reionización de centros metálicos
RI = Reionizer() #Neutraliza moléculas añadiendo o removiendo hidrógenos
TC = TautomerCanonicalizer()  #Devuelve un tautómero razonable, no es garantizado que sea el más favorable energéticamente

In [ ]:
#Función para curado
def pretreatment(smi):
    try:
        mol = Chem.MolFromSmiles(smi)
        if mol == None:
            #Si no se puede procesar el SMILES, indica "Error 1"
            return "Error 1"
        else:
            mol = STD(mol)
            mol = LFC(mol)

            allowed_elements = {"H","B","C","N","O","F","Si","P","S","Cl","Se","Br","I"} #Lista de elementos permitidos en la molécula, si aparece uno distinto se elimina la entrada
            actual_elements = set([atom.GetSymbol() for atom in mol.GetAtoms()])
            if len(actual_elements-allowed_elements) == 0:
                mol = UC(mol)
                mol = RI(mol)
                mol = TC(mol)
                return Chem.MolToSmiles(mol)
            else:
                #Si aparecen elementos no permitidos, indica "Error 2"
                return "Error 2"
    except:
        return "Error 3"

In [ ]:
#Crea una nueva columna con el resultado de aplicar la función de curado
df["CANONICAL_SMILES"] = [pretreatment(x) for x in df["SMILES"]]

In [ ]:
#Elimina SMILES no procesados por RDKit (Error 1)
df = df[df["CANONICAL_SMILES"] != "Error 1"]
#Elimina SMILES con átomos no permitidos (Error 2)
df = df[df["CANONICAL_SMILES"] != "Error 2"]
#Elimina otros errores
df = df[df["CANONICAL_SMILES"] != "Error 3"].reset_index(drop=True)

df.head()

,Unnamed: 0,NAME,SMILES,SOURCE_SPECIES,YEAR,REFERENCE,CANONICAL_SMILES
0,0,Gemin D,O=C(OC(C(=O)CO)C1OC(=O)c2cc(O)c(O)c(O)c2-c2c(c...,Phyllanthus acuminatus,2021.0,https://doi.org/10.3390/molecules26216493,O=C(OC(C(=O)CO)C1OC(=O)c2cc(O)c(O)c(O)c2-c2c(c...
1,1,Phyllanemblinin B,O=C(OC1OC(CO)C2OC(=O)c3cc(O)c(O)c(O)c3-c3c(cc(...,Phyllanthus acuminatus,2021.0,https://doi.org/10.3390/molecules26216493,O=C(OC1OC(CO)C2OC(=O)c3cc(O)c(O)c(O)c3-c3c(cc(...
2,2,Corilagin,O=C(OC1OC2COC(=O)c3cc(O)c(O)c(O)c3-c3c(cc(O)c(...,Phyllanthus acuminatus,2021.0,https://doi.org/10.3390/molecules26216493,O=C(OC1OC2COC(=O)c3cc(O)c(O)c(O)c3-c3c(cc(O)c(...
3,3,Prodelphinidin B dimer,Oc1cc(O)c2c(c1)OC(c1cc(O)c(O)c(O)c1)C(O)C2c1c(...,Phyllanthus acuminatus,2021.0,https://doi.org/10.3390/molecules26216493,Oc1cc(O)c2c(c1)OC(c1cc(O)c(O)c(O)c1)C(O)C2c1c(...
4,4,(epi)galocatequina,Oc1cc(O)c2c(c1)OC(c1cc(O)c(O)c(O)c1)C(O)C2,Phyllanthus acuminatus,2021.0,https://doi.org/10.3390/molecules26216493,Oc1cc(O)c2c(c1)OC(c1cc(O)c(O)c(O)c1)C(O)C2


##Generar IDs, eliminar duplicados y agrupar

In [ ]:
#Función para asignar los IDs de compuestos
def assign_compound_ID(input):
    unique_SMILES = df['CANONICAL_SMILES'].unique() #Lista de SMILES únicos
    compound_IDs = [i for i in range(1, len(unique_SMILES) + 1)] #Asigna un número a cada SMILES
    SMILES_dict = dict(zip(compound_IDs, unique_SMILES)) #Diccionario de SMILES con sus IDs
    for key, value in SMILES_dict.items(): #Para cada ID (key) y SMILES (value)
        if value == input: #Si el SMILES del diccionario coincide con el ingresado (desde el df)
            return key #Regresa su respectivo ID
    return None  #Si no encuentra el SMILES, regresa un valor nulo

df['COMPOUND_ID'] = df['CANONICAL_SMILES'].apply(assign_compound_ID) #Escribe el ID de cada compuesto en una nueva columna
df['COMPOUND_ID'] = df['COMPOUND_ID'].astype(int)

In [ ]:
#Función para asignar los IDs de fuentes, análoga a la de SMILES
def assign_source_ID(input):
    unique_source = df['SOURCE_SPECIES'].unique() #Lista de fuentes únicas
    source_IDs = [i for i in range(1, len(unique_source) + 1)] #Asigna un número a cada fuente
    source_dict = dict(zip(source_IDs, unique_source)) #Diccionario de fuentes con sus IDs
    for key, value in source_dict.items():
        if value == input:
            return key
    return None

df['SOURCE_ID'] = df['SOURCE_SPECIES'].apply(assign_source_ID)
df['SOURCE_ID'] = df['SOURCE_ID'].fillna(0).astype(int) #Asigna un cero si no hay información en la columna de SOURCE_SPECIES

In [ ]:
#Función para asignar los IDs de referencias
def assign_ref_ID(input):
    unique_ref = df.REFERENCE.unique()
    ref_IDs = [i for i in range(1, len(unique_ref) + 1)]
    ref_dict = dict(zip(ref_IDs, unique_ref))
    for key, value in ref_dict.items():
      if value == input:
        return key
    return None

df['REFERENCE_ID'] = df['REFERENCE'].apply(assign_ref_ID)
df['REFERENCE_ID'] = df['REFERENCE_ID'].astype(int)

In [ ]:
#Crear un ID global con los tres IDs generados
df['GLOBAL_ID'] = df['REFERENCE_ID'].astype(str) + '.' + df['SOURCE_ID'].astype(str) + '.' + df['COMPOUND_ID'].astype(str)

#Revisa y remueve filas duplicadas usando el ID global
duplicate_IDs = df[df.duplicated(subset='GLOBAL_ID', keep=False)]
df.drop_duplicates(subset='GLOBAL_ID', keep='first', inplace=True)

In [ ]:
#Agrupa las entradas del mismo compuesto en una sola fila, manteniendo todos los nombres, fuentes y referencias
df_grouped = df.groupby('COMPOUND_ID').agg({
    'CANONICAL_SMILES': 'first',
    'NAME': lambda x: list(x),
    'SOURCE_SPECIES': lambda x: list(x),
    'REFERENCE': lambda x: list(x)
}).reset_index()

df_grouped = df_grouped.rename(columns={'CANONICAL_SMILES': 'SMILES'})

df_grouped.head()

,COMPOUND_ID,SMILES,NAME,SOURCE_SPECIES,REFERENCE
0,1,O=C(OC(C(=O)CO)C1OC(=O)c2cc(O)c(O)c(O)c2-c2c(c...,[Gemin D],[Phyllanthus acuminatus],[https://doi.org/10.3390/molecules26216493]
1,2,O=C(OC1OC(CO)C2OC(=O)c3cc(O)c(O)c(O)c3-c3c(cc(...,[Phyllanemblinin B],[Phyllanthus acuminatus],[https://doi.org/10.3390/molecules26216493]
2,3,O=C(OC1OC2COC(=O)c3cc(O)c(O)c(O)c3-c3c(cc(O)c(...,[Corilagin],[Phyllanthus acuminatus],[https://doi.org/10.3390/molecules26216493]
3,4,Oc1cc(O)c2c(c1)OC(c1cc(O)c(O)c(O)c1)C(O)C2c1c(...,"[Prodelphinidin B dimer, PAC B-Type (E) GC-(E)GC]","[Phyllanthus acuminatus, Psidium friedrichstha...","[https://doi.org/10.3390/molecules26216493, ht..."
4,5,Oc1cc(O)c2c(c1)OC(c1cc(O)c(O)c(O)c1)C(O)C2,[(epi)galocatequina],[Phyllanthus acuminatus],[https://doi.org/10.3390/molecules26216493]


##Calcular descriptores

In [ ]:
!git clone https://github.com/AstraZeneca/peptide-tools.git #Copia el repositorio de peptide-tools para obtener la lista de subestructuras
substructures_smarts = '/content/peptide-tools/pIChemiSt/pichemist/data/smarts/legacy/smarts_pKaMatcher.dat' #Obtiene el archivo de subestructuras

Cloning into 'peptide-tools'...
remote: Enumerating objects: 1392, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1392 (delta 44), reused 53 (delta 42), pack-reused 1324 (from 1)
Receiving objects: 100% (1392/1392), 588.36 KiB | 9.49 MiB/s, done.
Resolving deltas: 100% (817/817), done.


In [ ]:
#Función para contar grupos ácidos y básicos
def ionizable_groups(SMILES):
  matches = {} #Diccionario de los centros de ionización. Traté de guardar los índices de los átomos para resaltarlos pero algunas de las posiciones definidas en el archivo de subestructuras fallan. Queda pendiente para el futuro
  compound_mol = Chem.MolFromSmiles(SMILES) #Convierte el SMILES en archivo Mol
  global ion_centers
  ion_centers = [] #Lista de centros de ionización

  with open(substructures_smarts, 'r') as f: #Abre el archivo de subestructuras
    for line in f.readlines(): #Lee el archivo línea por línea
      ln = line.strip().split() #Separa y obtiene la información separada por espacios
      if not ln or line.startswith('#'): continue #Salta líneas vacías o con comentarios

      substr_mol = Chem.MolFromSmarts(ln[1]) #Obtiene la subestructura de la línea como un archivo Mol
      substr_matches = compound_mol.GetSubstructMatches(substr_mol) #Tupla de coincidencias con la molécula ingresada (desde el df)

      if len(substr_matches) != 0: #Si hay al menos una coincidencia
        for substr in range(len(substr_matches)): #Para cada subestructura coincidente
          compound_center = substr_matches[substr][int(ln[2])-1] #Obtiene el índice del átomo en el centro de ionización
          if compound_center in ion_centers: #Si ya se incluyó a la lista de centros, se salta a la siguiente subestructura
            continue
          else: #Si no se ha incluído
            ion_centers.append(compound_center) #Se incluye a la lista de centros
            matches[compound_center] = ln[5] #Se añade al diccionario con el tipo de centro (acid/base)
            if len(ln) > 6: #Revisa si esta subestructura tiene más de un centro (e.j compuestos basados en catecoles) y lo añade al diccionario
              compound_center = substr_matches[substr][int(ln[6])-1]
              ion_centers.append(compound_center)
              matches[compound_center] = ln[9]

  total_acid = 0
  total_base = 0
  for key, value in matches.items(): #Para cada entrada en el diccionario de coincidencias
    if value == 'acid': #Si el centro es ácido
      total_acid += 1
    elif value == 'base': #Si el centro es básico
      total_base += 1
  group_count = (total_acid, total_base) #Tupla con las cuentas finales para la molécula

  return group_count

In [ ]:
#Aplica las funciones de RDKit para calcular los descriptores, más la de grupos ionizables definida en el bloque anterior
df_grouped['MW'] = df_grouped['SMILES'].apply(lambda m: Descriptors.MolWt(Chem.MolFromSmiles(m)))
df_grouped['HBA'] = df_grouped['SMILES'].apply(lambda m: Descriptors.NumHAcceptors(Chem.MolFromSmiles(m)))
df_grouped['HBD'] = df_grouped['SMILES'].apply(lambda m: Descriptors.NumHDonors(Chem.MolFromSmiles(m)))
df_grouped['RB'] = df_grouped['SMILES'].apply(lambda m: Descriptors.NumRotatableBonds(Chem.MolFromSmiles(m)))
df_grouped['AlogP'] = df_grouped['SMILES'].apply(lambda m: Descriptors.MolLogP(Chem.MolFromSmiles(m)))
df_grouped['TPSA'] = df_grouped['SMILES'].apply(lambda m: Descriptors.TPSA(Chem.MolFromSmiles(m)))
df_grouped['AG'] = df_grouped['SMILES'].apply(lambda x: ionizable_groups(x)[0])
df_grouped['BG'] = df_grouped['SMILES'].apply(lambda x: ionizable_groups(x)[1])

In [ ]:
df_grouped.head()

,COMPOUND_ID,SMILES,NAME,SOURCE_SPECIES,REFERENCE,MW,HBA,HBD,RB,AlogP,TPSA,AG,BG
0,1,O=C(OC(C(=O)CO)C1OC(=O)c2cc(O)c(O)c(O)c2-c2c(c...,[Gemin D],[Phyllanthus acuminatus],[https://doi.org/10.3390/molecules26216493],634.455,18,11,5,-0.4523,318.50,9,0
1,2,O=C(OC1OC(CO)C2OC(=O)c3cc(O)c(O)c(O)c3-c3c(cc(...,[Phyllanemblinin B],[Phyllanthus acuminatus],[https://doi.org/10.3390/molecules26216493],634.455,18,11,3,-0.2965,310.66,9,0
2,3,O=C(OC1OC2COC(=O)c3cc(O)c(O)c(O)c3-c3c(cc(O)c(...,[Corilagin],[Phyllanthus acuminatus],[https://doi.org/10.3390/molecules26216493],634.455,18,11,2,-0.2965,310.66,9,0
3,4,Oc1cc(O)c2c(c1)OC(c1cc(O)c(O)c(O)c1)C(O)C2c1c(...,"[Prodelphinidin B dimer, PAC B-Type (E) GC-(E)GC]","[Phyllanthus acuminatus, Psidium friedrichstha...","[https://doi.org/10.3390/molecules26216493, ht...",610.524,14,12,3,2.4062,261.22,11,0
4,5,Oc1cc(O)c2c(c1)OC(c1cc(O)c(O)c(O)c1)C(O)C2,[(epi)galocatequina],[Phyllanthus acuminatus],[https://doi.org/10.3390/molecules26216493],306.270,7,6,1,1.2517,130.61,5,0


##Guardar como CSV

In [ ]:
df_grouped.to_csv('file_processed.csv', index=False) #Cambiar nombre según sea necesario